In [1]:
import numpy as np 
import pandas as pd 
from itertools import product

In [2]:
df=pd.read_csv("../reports/welch_t_test/whole_comb_t_test.csv")

In [3]:
zone=['54ft_road','dvc_more','junction_mall','prantika_bus_stand','station']
timezone=['Early_Morning','Morning','Afternoon','Evening']

#axis creation
axis=list(product(zone,timezone))

In [4]:
#helper function
def get_pair_value(row,col):

    data_row_per1=df[(df.left_date=='*') & (df.right_date=='*') & (df.left_zone==row[0]) & (df.right_zone==col[0]) & (df.left_time_zone==row[1]) & (df.right_time_zone==col[1])]

    data_row_per2=df[(df.left_date=='*') & (df.right_date=='*') & (df.left_zone==col[0]) & (df.right_zone==row[0]) & (df.left_time_zone==col[1]) & (df.right_time_zone==row[1])]

    if data_row_per1.shape[0]>data_row_per2.shape[0]:
        data_row=data_row_per1.copy()
    elif data_row_per1.shape[0]<data_row_per2.shape[0]:
        data_row=data_row_per2.copy()
    else:
        data_row=data_row_per1.copy()

    if(row[0]==col[0] and row[1]==col[1]):
        val=0 #Pair values are same
    elif(data_row.dropna().shape[0]==0):
        val= 'N' #Pair is not possible
    elif(data_row.p_value.values[0]>=0.05):
        val= -1 #Pair is not significant
    elif(data_row.p_value.values[0]<0.05):
        val=+1 #Pair is significant
    return val

# Generating MATRIX

In [5]:
#MATRIX Creation
def get_matrix():
    matrix=[]
    for row in axis:
        ROW=[]
        for col in axis:
            value= get_pair_value(row,col)
            ROW.append(value)
        matrix.append(ROW[:])
    return np.array(matrix)

In [6]:
matrix=get_matrix()

# Saving Matrix

In [7]:
tz_map={'Early_Morning':"T1",'Morning':"T2",'Afternoon':"T3",'Evening':"T4"}


df_matrix=pd.DataFrame(data=matrix,
columns=pd.MultiIndex.from_tuples([(e[0],tz_map[e[1]])for e in axis]),
index=pd.MultiIndex.from_tuples([(e[0],tz_map[e[1]])for e in axis]))

In [11]:
df_matrix

54ft_road             dvc_more              \
                             T1  T2  T3  T4       T1  T2  T3  T4   
54ft_road          T1         0  -1  -1  -1        1   1  -1   1   
                   T2        -1   0  -1  -1       -1  -1  -1  -1   
                   T3        -1  -1   0  -1       -1  -1  -1  -1   
                   T4        -1  -1  -1   0       -1  -1  -1  -1   
dvc_more           T1         1  -1  -1  -1        0  -1  -1  -1   
                   T2         1  -1  -1  -1       -1   0  -1  -1   
                   T3        -1  -1  -1  -1       -1  -1   0  -1   
                   T4         1  -1  -1  -1       -1  -1  -1   0   
junction_mall      T1        -1  -1  -1  -1        1   1  -1   1   
                   T2        -1  -1  -1  -1       -1  -1  -1  -1   
                   T3        -1  -1  -1  -1       -1   1  -1  -1   
                   T4        -1  -1  -1  -1       -1  -1  -1  -1   
prantika_bus_stand T1         1   1   1   1        1  -1  -1  -1   
                   T2         1   1   1   1       -1  -1  -1  -1   
                   T3         1   1   1   1       -1  -1  -1  -1   
                   T4         1  -1  -1  -1       -1  -1  -1  -1   
station            T1         1   1  -1   1       -1  -1  -1  -1   
                   T2        -1  -1  -1  -1       -1  -1  -1  -1   
                   T3         1   1   1   1       -1  -1  -1  -1   
                   T4         N   N   N   N        N   N   N   N   

                      junction_mall             prantika_bus_stand          \
                                 T1  T2  T3  T4                 T1  T2  T3   
54ft_road          T1            -1  -1  -1  -1                  1   1   1   
                   T2            -1  -1  -1  -1                  1   1   1   
                   T3            -1  -1  -1  -1                  1   1   1   
                   T4            -1  -1  -1  -1                  1   1   1   
dvc_more           T1             1  -1  -1  -1                  1  -1  -1   
                   T2             1  -1   1  -1                 -1  -1  -1   
                   T3            -1  -1  -1  -1                 -1  -1  -1   
                   T4             1  -1  -1  -1                 -1  -1  -1   
junction_mall      T1             0  -1  -1  -1                  1   1   1   
                   T2            -1   0  -1  -1                  1   1   1   
                   T3            -1  -1   0  -1                  1   1   1   
                   T4            -1  -1  -1   0                  1  -1   1   
prantika_bus_stand T1             1   1   1   1                  0  -1  -1   
                   T2             1   1   1  -1                 -1   0  -1   
                   T3             1   1   1   1                 -1  -1   0   
                   T4             1  -1  -1  -1                 -1  -1  -1   
station            T1             1   1   1  -1                 -1  -1  -1   
                   T2            -1  -1  -1  -1                  1  -1  -1   
                   T3             1   1   1  -1                 -1  -1  -1   
                   T4             N   N   N   N                  N   N   N   

                          station             
                       T4      T1  T2  T3 T4  
54ft_road          T1   1       1  -1   1  N  
                   T2  -1       1  -1   1  N  
                   T3  -1      -1  -1   1  N  
                   T4  -1       1  -1   1  N  
dvc_more           T1  -1      -1  -1  -1  N  
                   T2  -1      -1  -1  -1  N  
                   T3  -1      -1  -1  -1  N  
                   T4  -1      -1  -1  -1  N  
junction_mall      T1   1       1  -1   1  N  
                   T2  -1       1  -1   1  N  
                   T3  -1       1  -1   1  N  
                   T4  -1      -1  -1  -1  N  
prantika_bus_stand T1  -1      -1   1  -1  N  
                   T2  -1      -1  -1  -1  N  
                   T3  -1      -1  -1  -1  N  
         

In [12]:
from pandas import ExcelWriter

writer = ExcelWriter('../reports/observ/matrix.xlsx')
df_matrix.to_excel(writer,'Sheet1')
writer.save()

In [13]:
#NICE